<a href="https://colab.research.google.com/github/Anushkadas0407/Anushkadas0407/blob/main/chapter_appendix-tools-for-deep-learning/jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from functools import lru_cache
from typing import List, Tuple

def rod_cut_max_revenue(prices: List[int], n: int) -> Tuple[int, List[int]]:
    """
    Top-down (recursive) rod cutting with memoization that also reconstructs the optimal cuts.

    Args:
        prices: prices[i] is the price of a rod of length i (1-indexed: prices[0] ignored).
        n: total rod length.

    Returns:
        (max_revenue, cuts) where cuts is a list of piece lengths that achieve max_revenue.
    """
    if n < 0:
        raise ValueError("Rod length n must be non-negative.")
    if not prices or len(prices) == 0:
        # No price info
        return (0, []) if n == 0 else (float("-inf"), [])
    if len(prices) <= n:
        raise ValueError(
            f"prices must include entries up to length {n}. "
            f"Got max length {len(prices) - 1}."
        )

    # Use -inf for impossible states; 0 revenue for length 0
    @lru_cache(maxsize=None)
    def f(len_left: int) -> int:
        if len_left == 0:
            return 0
        best = float("-inf")
        # try first cut of size i, then solve subproblem of len_left - i
        for i in range(1, len_left + 1):
            # prices is 1-indexed; prices[i] is price for a piece of length i
            best = max(best, prices[i] + f(len_left - i))
        return best

    # Reconstruct the optimal cuts by following the argmax choices
    def reconstruct(len_left: int) -> List[int]:
        cuts = []
        while len_left > 0:
            best_val = f(len_left)
            chosen_i = None
            for i in range(1, len_left + 1):
                if prices[i] + f(len_left - i) == best_val:
                    chosen_i = i
                    break
            cuts.append(chosen_i)
            len_left -= chosen_i
        return cuts

    return f(n), reconstruct(n)


# ---------- Demo / quick tests ----------
if __name__ == "__main__":
    # Classic CLRS example prices (1-indexed): p[1]=1, p[2]=5, ..., p[10]=30
    prices = [None, 1, 5, 8, 9, 10, 17, 17, 20, 24, 30]
    # Try different rod lengths
    for N in [1, 4, 7, 8, 10]:
        best, cuts = rod_cut_max_revenue(prices, N)
        print(f"n={N:>2}  ->  max revenue = {best:>2}, cuts = {cuts}")


n= 1  ->  max revenue =  1, cuts = [1]
n= 4  ->  max revenue = 10, cuts = [2, 2]
n= 7  ->  max revenue = 18, cuts = [1, 6]
n= 8  ->  max revenue = 22, cuts = [2, 6]
n=10  ->  max revenue = 30, cuts = [10]


In [11]:
from typing import List, Tuple

def matrix_chain_min_cost(dims: List[int]) -> Tuple[int, List[List[int]], List[List[int]], str]:

    n = len(dims) - 1
    if n <= 0:
        return 0, [], [], ""

    # m[i][j] = minimal cost to multiply Ai..Aj
    # s[i][j] = index k where we split at optimal solution for i..j
    m = [[0]*(n+1) for _ in range(n+1)]
    s = [[0]*(n+1) for _ in range(n+1)]

    # chain_len = length of subchain
    for chain_len in range(2, n+1):
        for i in range(1, n - chain_len + 2):
            j = i + chain_len - 1
            m[i][j] = float('inf')
            # try all possible first split positions
            for k in range(i, j):
                cost = (
                    m[i][k] +
                    m[k+1][j] +
                    dims[i-1] * dims[k] * dims[j]
                )
                if cost < m[i][j]:
                    m[i][j] = cost
                    s[i][j] = k

    def build_parens(i: int, j: int) -> str:
        if i == j:
            return f"A{i}"
        k = s[i][j]
        left = build_parens(i, k)
        right = build_parens(k+1, j)
        return f"({left}×{right})"

    parens = build_parens(1, n)
    return m[1][n], m, s, parens


# ---------- Demo / quick tests ----------
if __name__ == "__main__":
    # Classic CLRS example
    dims = [30, 35, 15, 5, 10, 20, 25]   # 6 matrices: A1..A6
    min_cost, m, s, parens = matrix_chain_min_cost(dims)
    print("Dimensions:", dims)
    print("Minimal cost:", min_cost)      # Expected: 15125
    print("Optimal parenthesization:", parens)

    # You can try smaller custom tests too
    dims2 = [10, 30, 5, 60]               # A1(10x30), A2(30x5), A3(5x60)
    min_cost2, _, _, parens2 = matrix_chain_min_cost(dims2)
    print("\nDimensions:", dims2)
    print("Minimal cost:", min_cost2)     # Expected: 4500
    print("Optimal parenthesization:", parens2)


Dimensions: [30, 35, 15, 5, 10, 20, 25]
Minimal cost: 15125
Optimal parenthesization: ((A1×(A2×A3))×((A4×A5)×A6))

Dimensions: [10, 30, 5, 60]
Minimal cost: 4500
Optimal parenthesization: ((A1×A2)×A3)


In [3]:
def lcs(X: str, Y: str):
    m, n = len(X), len(Y)

    # DP Table (m+1 x n+1)
    dp = [[0]*(n+1) for _ in range(m+1)]

    # Fill DP bottom-up
    for i in range(1, m+1):
        for j in range(1, n+1):
            if X[i-1] == Y[j-1]:
                dp[i][j] = 1 + dp[i-1][j-1]
            else:
                dp[i][j] = max(dp[i-1][j], dp[i][j-1])

    # Reconstruct LCS string
    i, j = m, n
    lcs_str = []
    while i > 0 and j > 0:
        if X[i-1] == Y[j-1]:
            lcs_str.append(X[i-1])
            i -= 1
            j -= 1
        elif dp[i-1][j] > dp[i][j-1]:
            i -= 1
        else:
            j -= 1

    return dp[m][n], ''.join(reversed(lcs_str))


# Example test
X = "AGGTAB"
Y = "GXTXAYB"
length, lcs_str = lcs(X, Y)
print("String 1:", X)
print("String 2:", Y)
print("LCS length:", length)
print("LCS:", lcs_str)


String 1: AGGTAB
String 2: GXTXAYB
LCS length: 4
LCS: GTAB


In [4]:
from typing import List, Tuple

def knapsack_01(weights: List[int], values: List[int], W: int) -> Tuple[int, List[int]]:
    """
    0/1 Knapsack (bottom-up DP) with reconstruction of chosen item indices.

    Args:
        weights: weights[i] is the weight of item i
        values:  values[i] is the value  of item i
        W:       capacity of the knapsack

    Returns:
        max_value, chosen_indices  (indices are 0-based)
    """
    n = len(weights)
    if n != len(values):
        raise ValueError("weights and values must be the same length")
    if W < 0:
        raise ValueError("Capacity W must be non-negative")

    # dp[i][w] = max value using first i items (0..i-1) with capacity w
    dp = [[0]*(W + 1) for _ in range(n + 1)]

    for i in range(1, n + 1):
        wt, val = weights[i-1], values[i-1]
        for w in range(0, W + 1):
            # Not take item i-1
            dp[i][w] = dp[i-1][w]
            # Take item i-1 (if it fits)
            if wt <= w:
                dp[i][w] = max(dp[i][w], val + dp[i-1][w - wt])

    # Reconstruct chosen items
    chosen = []
    w = W
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i-1][w]:       # item i-1 was taken
            chosen.append(i-1)
            w -= weights[i-1]
    chosen.reverse()
    return dp[n][W], chosen


def knapsack_01_optimized(weights: List[int], values: List[int], W: int) -> int:
    """
    Space-optimized 0/1 knapsack using 1D DP.
    Returns only the maximum value (no reconstruction).

    dp[w] = best value achievable with capacity w using items seen so far.
    Iterate capacities in reverse to enforce 0/1 (use-at-most-once).
    """
    n = len(weights)
    if n != len(values):
        raise ValueError("weights and values must be the same length")
    if W < 0:
        raise ValueError("Capacity W must be non-negative")

    dp = [0]*(W + 1)
    for wt, val in zip(weights, values):
        for w in range(W, wt - 1, -1):   # reverse to avoid reusing the same item
            dp[w] = max(dp[w], val + dp[w - wt])
    return dp[W]


# ---------- Demo / quick tests ----------
if __name__ == "__main__":
    weights = [2, 3, 4, 5]
    values  = [3, 4, 5, 8]
    W = 8

    best, chosen = knapsack_01(weights, values, W)
    print("2D DP -> max value:", best, "chosen items (0-based):", chosen)

    best_opt = knapsack_01_optimized(weights, values, W)
    print("1D DP -> max value:", best_opt)



2D DP -> max value: 12 chosen items (0-based): [1, 3]
1D DP -> max value: 12


In [1]:
from typing import List, Tuple

def select_max_activities(activities: List[Tuple[int, int, str]]):
    """
    Select a maximum-size subset of mutually compatible activities (non-overlapping),
    using the classic greedy algorithm: sort by earliest finish time.

    Args:
        activities: list of (start, finish, label)
                    Convention: half-open intervals [start, finish)
                    i.e., an activity that ends at time t is compatible with one that starts at time t.

    Returns:
        chosen: list of selected (start, finish, label) in the order chosen
    """
    if not activities:
        return []

    # 1) sort by finish time
    acts = sorted(activities, key=lambda x: x[1])

    # 2) greedily pick compatible ones
    chosen = []
    last_finish = float("-inf")
    for s, f, lbl in acts:
        if s >= last_finish:     # half-open: compatible if next start >= last_finish
            chosen.append((s, f, lbl))
            last_finish = f
    return chosen


# ---------- Demo / quick tests ----------
if __name__ == "__main__":
    # Example activities: (start, finish, label)
    activities = [
        (1, 4,  "A"),
        (3, 5,  "B"),
        (0, 6,  "C"),
        (5, 7,  "D"),
        (3, 9,  "E"),
        (5, 9,  "F"),
        (6, 10, "G"),
        (8, 11, "H"),
        (8, 12, "I"),
        (2, 14, "J"),
        (12,16, "K")
    ]
    chosen = select_max_activities(activities)
    print("Chosen activities (start, finish, label) in selection order:")
    for a in chosen:
        print(a)


Chosen activities (start, finish, label) in selection order:
(1, 4, 'A')
(5, 7, 'D')
(8, 11, 'H')
(12, 16, 'K')


In [2]:
from typing import List, Tuple

def fractional_knapsack(weights: List[float], values: List[float], W: float) -> Tuple[float, List[Tuple[int, float]]]:
    """
    Greedy fractional knapsack.

    Args:
        weights: weights[i] of item i
        values:  values[i]  of item i
        W:       capacity of knapsack (can be float)

    Returns:
        (max_value, picks)
        - max_value: maximal achievable value (float)
        - picks: list of (item_index, fraction_taken) where 0 < fraction_taken <= 1
                 item_index is 0-based
    """


Max value: 240.0
Picks (index, fraction): [(0, 1.0), (1, 1.0), (2, 0.6666666666666666)]


In [4]:
import heapq
from typing import List, Tuple, Dict

def dijkstra(graph: Dict[int, List[Tuple[int, int]]], source: int) -> Dict[int, int]:
    """
    graph: adjacency list form -> graph[u] = list of (v, weight)
    source: starting vertex

    returns shortest distance to every vertex
    """

    # distance dictionary init with infinite distance
    dist = {node: float('inf') for node in graph}
    dist[source] = 0

    # min heap priority queue (distance, node)
    pq = [(0, source)]

    while pq:
        current_dist, u = heapq.heappop(pq)

        # skip if outdated
        if current_dist > dist[u]:
            continue

        for v, w in graph[u]:
            if dist[u] + w < dist[v]:
                dist[v] = dist[u] + w
                heapq.heappush(pq, (dist[v], v))

    return dist


# -------- Demo --------
if __name__ == "__main__":
    # graph: u -> (v, weight)
    graph = {
        0: [(1, 4), (2, 1)],
        1: [(3, 1)],
        2: [(1, 2), (3, 5)],
        3: []
    }
    source = 0
    print("Shortest Distances from source:", source)
    print(dijkstra(graph, source))





Shortest Distances from source: 0
{0: 0, 1: 3, 2: 1, 3: 4}


In [5]:
from typing import List, Tuple

class DSU:
    def __init__(self, n: int):
        self.parent = list(range(n))
        self.rank = [0]*n

    def find(self, x: int) -> int:
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])  # path compression
        return self.parent[x]

    def union(self, a: int, b: int) -> bool:
        ra, rb = self.find(a), self.find(b)
        if ra == rb:
            return False
        # union by rank
        if self.rank[ra] < self.rank[rb]:
            self.parent[ra] = rb
        elif self.rank[rb] < self.rank[ra]:
            self.parent[rb] = ra
        else:
            self.parent[rb] = ra
            self.rank[ra] += 1
        return True

def kruskal_mst(n: int, edges: List[Tuple[int, int, int]]):
    """
    Kruskal's MST
    n     : number of vertices labeled 0..n-1
    edges : list of (u, v, w) for undirected weighted edges
    Returns: (total_cost, mst_edges)
    """
    # sort edges by weight
    edges_sorted = sorted(edges, key=lambda e: e[2])
    dsu = DSU(n)
    mst_cost = 0
    mst_edges = []

    for u, v, w in edges_sorted:
        if dsu.union(u, v):
            mst_cost += w
            mst_edges.append((u, v, w))
            if len(mst_edges) == n-1:  # early stop once MST formed
                break
    return mst_cost, mst_edges

# ---- Demo (Kruskal) ----
if __name__ == "__main__":
    n = 6
    edges = [
        (0,1,4),(0,2,4),(1,2,2),(2,3,3),
        (2,4,2),(2,5,4),(3,5,2),(4,5,3)
    ]
    cost, mst = kruskal_mst(n, edges)
    print("Kruskal -> MST cost:", cost)
    print("Edges:", mst)


Kruskal -> MST cost: 13
Edges: [(1, 2, 2), (2, 4, 2), (3, 5, 2), (2, 3, 3), (0, 1, 4)]


In [8]:

import heapq
from typing import Dict, List, Tuple

def prim_mst(graph: Dict[int, List[Tuple[int,int]]], start: int = 0):
    """
    Prim's MST using a min-heap.
    graph[u] = list of (v, w) undirected neighbors with weight w
    start: starting vertex

    Returns: (total_cost, mst_edges)
    """
    visited = set()
    pq = []
    mst_edges = []
    total_cost = 0

    def push_edges(u: int):
        visited.add(u)
        for v, w in graph[u]:
            if v not in visited:
                heapq.heappush(pq, (w, u, v))

    # Initialize with start node's edges
    push_edges(start)

    while pq and len(visited) < len(graph):
        w, u, v = heapq.heappop(pq)
        if v in visited:
            continue
        mst_edges.append((u, v, w))
        total_cost += w
        push_edges(v)

    # If graph is disconnected, this returns an MST for the connected component of 'start'
    return total_cost, mst_edges

# ---- Demo (Prim) ----
if __name__ == "__main__":
    graph = {
        0: [(1,4),(2,4)],
        1: [(0,4),(2,2)],
        2: [(0,4),(1,2),(3,3),(4,2),(5,4)],
        3: [(2,3),(5,2)],
        4: [(2,2),(5,3)],
        5: [(2,4),(3,2),(4,3)]
    }
    cost, mst = prim_mst(graph, start=0)
    print("Prim  -> MST cost:", cost)
    print("Edges:", mst)


Prim  -> MST cost: 13
Edges: [(0, 1, 4), (1, 2, 2), (2, 4, 2), (2, 3, 3), (3, 5, 2)]


In [9]:
from typing import List, Optional, Tuple

INF = float("inf")

def floyd_warshall(dist: List[List[float]]) -> Tuple[List[List[float]], List[List[Optional[int]]], bool]:
    """
    Floyd–Warshall APSP with path reconstruction and negative-cycle detection.

    Args:
        dist: n x n matrix where dist[i][j] is the edge weight i->j.
              Use 0 on the diagonal, INF if no direct edge.

    Returns:
        D:    n x n matrix of shortest-path distances
        nxt:  n x n matrix for path reconstruction; nxt[i][j] = next node after i on a shortest path to j (or None)
        has_neg_cycle: True if the graph contains a negative cycle reachable between some pair

    Notes:
        - Works with negative edges as long as there is no negative cycle affecting the path.
        - To reconstruct path i->j, use the 'reconstruct_path' helper below.
    """
    n = len(dist)
    # Copy input to avoid in-place surprises
    D = [row[:] for row in dist]
    nxt: List[List[Optional[int]]] = [[None]*n for _ in range(n)]

    # Initialize nxt for edges that exist
    for i in range(n):
        for j in range(n):
            if i != j and D[i][j] < INF:
                nxt[i][j] = j

    # Main triple loop
    for k in range(n):
        for i in range(n):
            Dik = D[i][k]
            if Dik == INF:  # small pruning
                continue
            for j in range(n):
                alt = Dik + D[k][j]
                if alt < D[i][j]:
                    D[i][j] = alt
                    nxt[i][j] = nxt[i][k]

    # Negative-cycle check: if any D[i][i] < 0, that vertex lies on a negative cycle
    has_neg_cycle = any(D[v][v] < 0 for v in range(n))
    return D, nxt, has_neg_cycle


def reconstruct_path(i: int, j: int, nxt: List[List[Optional[int]]]) -> List[int]:
    """
    Reconstruct one shortest path from i to j using the 'nxt' matrix.
    Returns empty list if no path.
    """
    if nxt[i][j] is None:
        return []
    path = [i]
    while i != j:
        i = nxt[i][j]
        if i is None:  # safety
            return []
        path.append(i)
    return path


# ---------- Demo / quick tests ----------
if __name__ == "__main__":
    # Example graph (directed). Use INF where there is no direct edge.
    #   0 ->1 (3), 0 ->2 (8), 0 ->4 (-4)
    #   1 ->3 (1), 1 ->4 (7)
    #   2 ->1 (4)
    #   3 ->0 (2), 3 ->2 (-5)
    #   4 ->3 (6)
    INF = float("inf")
    dist = [
        [0,   3,   8,   INF, -4],
        [INF, 0,   INF, 1,    7 ],
        [INF, 4,   0,   INF, INF],
        [2,   INF, -5,  0,   INF],
        [INF, INF, INF, 6,    0 ]
    ]

    D, nxt, neg = floyd_warshall(dist)
    print("Has negative cycle?", neg)
    print("All-pairs shortest distances:")
    for row in D:
        print(row)

    # Reconstruct example path 0 -> 3
    p = reconstruct_path(0, 3, nxt)
    print("Path 0->3:", p)


Has negative cycle? False
All-pairs shortest distances:
[0, 1, -3, 2, -4]
[3, 0, -4, 1, -1]
[7, 4, 0, 5, 3]
[2, -1, -5, 0, -2]
[8, 5, 1, 6, 0]
Path 0->3: [0, 4, 3]


In [10]:
from typing import Dict, List, Tuple, Set

def tarjan_scc(graph: Dict[int, List[int]]) -> List[List[int]]:
    """
    Tarjan's algorithm to find all SCCs in a directed graph.
    graph[u] -> list of outgoing neighbors v

    Returns:
        List of SCCs, each SCC is a list of vertices (in discovery order within that SCC).
    """
    index = { }              # discovery index per node
    low = { }                # low-link value per node
    on_stack = set()         # nodes currently in the DFS stack
    stack: List[int] = []
    sccs: List[List[int]] = []
    time = 0

    def strongconnect(u: int):
        nonlocal time
        index[u] = time
        low[u] = time
        time += 1
        stack.append(u)
        on_stack.add(u)

        # DFS on neighbors
        for v in graph.get(u, []):
            if v not in index:
                strongconnect(v)
                low[u] = min(low[u], low[v])
            elif v in on_stack:
                low[u] = min(low[u], index[v])

        # If u is a root node, pop the stack and generate an SCC
        if low[u] == index[u]:
            comp = []
            while True:
                w = stack.pop()
                on_stack.remove(w)
                comp.append(w)
                if w == u:
                    break
            sccs.append(comp)

    # Run DFS from every node (covers disconnected components)
    for u in graph.keys():
        if u not in index:
            strongconnect(u)

    return sccs


def condensation_dag(graph: Dict[int, List[int]], sccs: List[List[int]]) -> Dict[int, Set[int]]:
    """
    Build the DAG of SCCs (also called the component graph / meta-graph).
    Returns:
        comp_graph: adjacency as comp_id -> set(next_comp_ids)
        where comp_id is an index into sccs list.
    """
    # Map node -> component id
    comp_id = {}
    for cid, comp in enumerate(sccs):
        for u in comp:
            comp_id[u] = cid

    comp_graph: Dict[int, Set[int]] = {cid: set() for cid in range(len(sccs))}
    for u, nbrs in graph.items():
        cu = comp_id[u]
        for v in nbrs:
            cv = comp_id[v]
            if cu != cv:
                comp_graph[cu].add(cv)
    return comp_graph


# ---------- Demo / quick tests ----------
if __name__ == "__main__":
    # Example 1 (classic):
    # 0→1, 1→2, 2→0 (SCC {0,1,2})
    # 1→3
    # 3→4, 4→5, 5→3 (SCC {3,4,5})
    # 2→6, 6→7, 7→8, 8→6 (SCC {6,7,8})
    # 8→9
    graph1 = {
        0:[1],
        1:[2,3],
        2:[0,6],
        3:[4],
        4:[5],
        5:[3],
        6:[7],
        7:[8],
        8:[6,9],
        9:[]
    }

    sccs1 = tarjan_scc(graph1)
    print("SCCs (example 1):", sccs1)

    comp_graph1 = condensation_dag(graph1, sccs1)
    print("Condensation DAG (as adjacency sets):", comp_graph1)

    # Example 2 (singletons + one 2-cycle)
    graph2 = {
        0:[1],
        1:[],
        2:[3],
        3:[2],
        4:[]
    }
    sccs2 = tarjan_scc(graph2)
    print("\nSCCs (example 2):", sccs2)


SCCs (example 1): [[9], [8, 7, 6], [5, 4, 3], [2, 1, 0]]
Condensation DAG (as adjacency sets): {0: set(), 1: {0}, 2: set(), 3: {1, 2}}

SCCs (example 2): [[1], [0], [3, 2], [4]]
